# Structured Persona Configuration and Dynamic Selection

In this notebook, we will explore how to build AI agent personas using structured configuration with Pydantic models. While basic personas can be defined with simple system prompts, production AI agents require more robust approaches that enable dynamic persona selection, consistent behavior across interactions, and adaptation based on user context.

**Why structured persona configuration matters:**
- **Consistency**: Ensures predictable agent behavior across interactions
- **Maintainability**: Makes persona definitions easy to update and version
- **Validation**: Catches configuration errors before deployment
- **Scalability**: Supports multiple personas in a single system
- **Adaptability**: Enables dynamic persona selection based on context

In [1]:
from enum import Enum
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
import logging
import os
from copy import deepcopy
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Retrieve OpenAI API key from environment
openai_api_key = os.getenv("OPENAI_API_KEY", "").strip()

## Understanding persona attributes
A well-defined AI agent persona consists of several key attributes that shape how the agent communicates and behaves. Let's break down these attributes one by one. Core persona attributes:
1. Expertise level: How knowledgeable the agent appears (beginner, intermediate, expert)
2. Communication style: How the agent expresses information (formal, casual, technical)
3. Tone: The emotional quality of responses (friendly, professional, empathetic)
4. Values: Principles the agent prioritizes (accuracy, helpfulness, efficiency)
5. Constraints: Boundaries the agent must respect (no medical advice, no financial recommendations)

### Defining expertise levels with enums
Let's start by defining expertise levels using Python enums. Enums provide a controlled vocabulary that prevents invalid values.

In [2]:
class ExpertiseLevel(str, Enum):
    """Defines the expertise level of the AI agent persona."""
    BEGINNER = "beginner"
    INTERMEDIATE = "intermediate"
    EXPERT = "expert"
    ADAPTIVE = "adaptive"  # Adjusts based on user level

This enum defines four expertise levels:
- `BEGINNER`: Uses simple language, provides detailed explanations, avoids jargon
- `INTERMEDIATE`: Balances technical terms with explanations, assumes some background knowledge
- `EXPERT`: Uses technical terminology freely, focuses on advanced concepts
- `ADAPTIVE`: Dynamically adjusts based on user's demonstrated expertise

By inheriting from both `str` and `Enum`, we ensure the values are strings that can be easily serialized to JSON while maintaining type safety.

### Defining communication styles
Next, let's define communication styles that determine how the agent expresses information.

In [3]:
class CommunicationStyle(str, Enum):
    """Defines how the agent communicates information."""
    FORMAL = "formal"
    CASUAL = "casual"
    TECHNICAL = "technical"
    CONVERSATIONAL = "conversational"
    EDUCATIONAL = "educational"

Each communication style shapes the agent's language:
- `FORMAL`: Professional language, complete sentences, proper grammar
- `CASUAL`: Relaxed tone, contractions, friendly expressions
- `TECHNICAL`: Precise terminology, detailed specifications, industry jargon
- `CONVERSATIONAL`: Natural dialogue, questions, engagement
- `EDUCATIONAL`: Clear explanations, examples, step-by-step guidance

### Defining tone attributes
Tone attributes define the emotional quality of the agent's responses.

In [4]:
class ToneAttribute(str, Enum):
    """Defines the emotional quality of agent responses."""
    FRIENDLY = "friendly"
    PROFESSIONAL = "professional"
    EMPATHETIC = "empathetic"
    ENTHUSIASTIC = "enthusiastic"
    PATIENT = "patient"
    DIRECT = "direct"

Tone attributes can be combined to create nuanced personas:
- `FRIENDLY`: Warm, approachable, uses positive language
- `PROFESSIONAL`: Maintains boundaries, focuses on task completion
- `EMPATHETIC`: Acknowledges emotions, shows understanding
- `ENTHUSIASTIC`: Expresses excitement, uses exclamation points
- `PATIENT`: Takes time to explain, doesn't rush
- `DIRECT`: Gets to the point quickly, minimal small talk

## Building a structured persona configuration

### Creating the base persona model
Now let's build a Pydantic model that combines all these attributes into a structured persona configuration.

In [5]:
class PersonaConfig(BaseModel):
    """Structured configuration for an AI agent persona."""
    
    # Identity
    name: str = Field(description="Name of the persona")
    role: str = Field(description="Role or job title of the persona")
    
    # Expertise and Communication
    expertise_level: ExpertiseLevel = Field(description="Level of expertise")
    communication_style: CommunicationStyle = Field(description="How the agent communicates")
    tone_attributes: List[ToneAttribute] = Field(description="Emotional qualities of responses")

This base model defines the core identity and communication attributes:
- `name`: A human-readable identifier for the persona (e.g., "Technical Support Agent")
- `role`: The persona's job or function (e.g., "Customer Service Representative")
- `expertise_level`: Uses our ExpertiseLevel enum for type safety
- `communication_style`: Uses our CommunicationStyle enum
- `tone_attributes`: A list of ToneAttribute enums, allowing multiple tones to be combined

The `Field()` function provides descriptions that help with documentation and validation.

### Adding domain knowledge
Let's extend our persona model to include domain-specific knowledge areas.

In [6]:
class PersonaConfig(BaseModel):
    """Structured configuration for an AI agent persona."""
    
    # Identity
    name: str = Field(description="Name of the persona")
    role: str = Field(description="Role or job title of the persona")
    
    # Expertise and Communication
    expertise_level: ExpertiseLevel = Field(description="Level of expertise")
    communication_style: CommunicationStyle = Field(description="How the agent communicates")
    tone_attributes: List[ToneAttribute] = Field(description="Emotional qualities of responses")
    
    # Domain Knowledge
    domain_knowledge: List[str] = Field(
        description="Areas of expertise and knowledge domains",
        default_factory=list
    )
    language_complexity: str = Field(
        description="Complexity of language used",
        default="moderate"
    )

We have added two new fields:
- `domain_knowledge`: A list of knowledge areas (e.g., ["Python programming", "Machine learning", "Data analysis"])
- `language_complexity`: Describes vocabulary level ("simple", "moderate", "advanced")

The `default_factory=list` ensures each persona gets its own empty list if no domains are specified, and `default="moderate"` provides a sensible default for language complexity.

### Adding values and constraints
Now let's add the values the persona prioritizes and constraints it must respect.

In [7]:
class PersonaConfig(BaseModel):
    """Structured configuration for an AI agent persona."""
    
    # Identity
    name: str = Field(description="Name of the persona")
    role: str = Field(description="Role or job title of the persona")
    
    # Expertise and Communication
    expertise_level: ExpertiseLevel = Field(description="Level of expertise")
    communication_style: CommunicationStyle = Field(description="How the agent communicates")
    tone_attributes: List[ToneAttribute] = Field(description="Emotional qualities of responses")
    
    # Domain Knowledge
    domain_knowledge: List[str] = Field(
        description="Areas of expertise and knowledge domains",
        default_factory=list
    )
    language_complexity: str = Field(
        description="Complexity of language used",
        default="moderate"
    )
    
    # Values and Constraints
    core_values: List[str] = Field(
        description="Principles the persona prioritizes",
        default_factory=lambda: ["accuracy", "helpfulness"]
    )
    constraints: List[str] = Field(
        description="Boundaries and limitations the persona must respect",
        default_factory=list
    )
    response_length: str = Field(
        description="Preferred response length",
        default="moderate"
    )

We have added three important fields:
- `core_values`: Principles that guide the persona's behavior (e.g., ["accuracy", "empathy", "efficiency"])
- `constraints`: Explicit boundaries (e.g., ["No medical advice", "No financial recommendations"])
- `response_length`: Preferred verbosity ("concise", "moderate", "detailed")

The `default_factory=lambda: ["accuracy", "helpfulness"]` provides default values that apply to most personas.

### Converting persona to system prompt
Now let's add a method that converts our structured persona into a system prompt for the LLM.

In [8]:
class PersonaConfig(BaseModel):
    """Structured configuration for an AI agent persona."""
    
    # Identity
    name: str = Field(description="Name of the persona")
    role: str = Field(description="Role or job title of the persona")
    
    # Expertise and Communication
    expertise_level: ExpertiseLevel = Field(description="Level of expertise")
    communication_style: CommunicationStyle = Field(description="How the agent communicates")
    tone_attributes: List[ToneAttribute] = Field(description="Emotional qualities of responses")
    
    # Domain Knowledge
    domain_knowledge: List[str] = Field(
        description="Areas of expertise and knowledge domains",
        default_factory=list
    )
    language_complexity: str = Field(
        description="Complexity of language used",
        default="moderate"
    )
    
    # Values and Constraints
    core_values: List[str] = Field(
        description="Principles the persona prioritizes",
        default_factory=lambda: ["accuracy", "helpfulness"]
    )
    constraints: List[str] = Field(
        description="Boundaries and limitations the persona must respect",
        default_factory=list
    )
    response_length: str = Field(
        description="Preferred response length",
        default="moderate"
    )
    
    def to_system_prompt(self) -> str:
        """Convert persona configuration to a system prompt."""
        prompt_parts = []
        
        # Identity
        prompt_parts.append(f"You are {self.name}, a {self.role}.")
        
        # Expertise
        prompt_parts.append(f"Your expertise level is {self.expertise_level.value}.")
        
        # Communication style
        prompt_parts.append(f"You communicate in a {self.communication_style.value} style.")
        
        # Tone
        if self.tone_attributes:
            tones = ", ".join([tone.value for tone in self.tone_attributes])
            prompt_parts.append(f"Your tone is {tones}.")
        
        # Domain knowledge
        if self.domain_knowledge:
            domains = ", ".join(self.domain_knowledge)
            prompt_parts.append(f"Your areas of expertise include: {domains}.")
        
        # Values
        if self.core_values:
            values = ", ".join(self.core_values)
            prompt_parts.append(f"You prioritize: {values}.")
        
        # Constraints
        if self.constraints:
            constraints = "\n- ".join(self.constraints)
            prompt_parts.append(f"You must respect these constraints:\n- {constraints}")
        
        # Response length
        prompt_parts.append(f"Provide {self.response_length} responses.")
        
        return "\n\n".join(prompt_parts)

The `to_system_prompt()` method constructs a natural language system prompt from the structured configuration:
1. Starts with identity (name and role)
2. Describes expertise level and communication style
3. Lists tone attributes
4. Enumerates domain knowledge areas
5. States core values
6. Lists constraints as bullet points
7. Specifies preferred response length

Each section is separated by double newlines for readability. The method uses `.value` to extract the string value from enum fields.

## Creating persona examples

### Example 1: Educational tutor persona
Let's create a persona for an educational tutor that adapts to student level.

In [9]:
# Create an educational tutor persona
educational_tutor = PersonaConfig(
    name="Alex the Tutor",
    role="Educational AI Tutor",
    expertise_level=ExpertiseLevel.ADAPTIVE,
    communication_style=CommunicationStyle.EDUCATIONAL,
    tone_attributes=[ToneAttribute.FRIENDLY, ToneAttribute.PATIENT, ToneAttribute.ENTHUSIASTIC],
    domain_knowledge=["Mathematics", "Science", "Programming", "Study Skills"],
    language_complexity="adaptive",
    core_values=["clarity", "encouragement", "accuracy", "student success"],
    constraints=[
        "Never provide direct answers to homework problems",
        "Always encourage critical thinking",
        "Adapt explanations to student's demonstrated level"
    ],
    response_length="detailed"
)

# Generate system prompt
tutor_prompt = educational_tutor.to_system_prompt()
print(tutor_prompt)

You are Alex the Tutor, a Educational AI Tutor.

Your expertise level is adaptive.

You communicate in a educational style.

Your tone is friendly, patient, enthusiastic.

Your areas of expertise include: Mathematics, Science, Programming, Study Skills.

You prioritize: clarity, encouragement, accuracy, student success.

You must respect these constraints:
- Never provide direct answers to homework problems
- Always encourage critical thinking
- Adapt explanations to student's demonstrated level

Provide detailed responses.


This persona is designed for educational contexts:
- `ADAPTIVE` expertise level allows adjusting to student knowledge
- `EDUCATIONAL` communication style focuses on teaching
- Multiple tone attributes create a supportive learning environment
- Broad domain knowledge covers common student needs
- Constraints ensure the tutor guides rather than just providing answers
- `detailed` responses provide thorough explanations

### Example 2: Technical support agent
Now let's create a persona for technical customer support.

In [10]:
# Create a technical support persona
tech_support = PersonaConfig(
    name="TechHelper",
    role="Technical Support Specialist",
    expertise_level=ExpertiseLevel.EXPERT,
    communication_style=CommunicationStyle.TECHNICAL,
    tone_attributes=[ToneAttribute.PROFESSIONAL, ToneAttribute.PATIENT, ToneAttribute.DIRECT],
    domain_knowledge=["Software troubleshooting", "Hardware diagnostics", "Network issues", "Security"],
    language_complexity="moderate",
    core_values=["efficiency", "accuracy", "customer satisfaction"],
    constraints=[
        "Never ask for sensitive credentials",
        "Always verify user's technical level before providing solutions",
        "Escalate complex issues appropriately"
    ],
    response_length="moderate"
)

# Generate system prompt
support_prompt = tech_support.to_system_prompt()
print(support_prompt)

You are TechHelper, a Technical Support Specialist.

Your expertise level is expert.

You communicate in a technical style.

Your tone is professional, patient, direct.

Your areas of expertise include: Software troubleshooting, Hardware diagnostics, Network issues, Security.

You prioritize: efficiency, accuracy, customer satisfaction.

You must respect these constraints:
- Never ask for sensitive credentials
- Always verify user's technical level before providing solutions
- Escalate complex issues appropriately

Provide moderate responses.


This persona is optimized for technical support:
- `EXPERT` level allows using technical terminology
- `TECHNICAL` communication style provides precise instructions
- `PROFESSIONAL` and `DIRECT` tones balance courtesy with efficiency
- Domain knowledge covers common support areas
- Constraints ensure security and appropriate escalation
- `moderate` response length balances detail with efficiency

### Example 3: Customer service representative
Let's create a persona for general customer service.

In [11]:
# Create a customer service persona
customer_service = PersonaConfig(
    name="ServicePro",
    role="Customer Service Representative",
    expertise_level=ExpertiseLevel.INTERMEDIATE,
    communication_style=CommunicationStyle.CONVERSATIONAL,
    tone_attributes=[ToneAttribute.FRIENDLY, ToneAttribute.EMPATHETIC, ToneAttribute.PROFESSIONAL],
    domain_knowledge=["Product information", "Order processing", "Returns and refunds", "Account management"],
    language_complexity="simple",
    core_values=["customer satisfaction", "empathy", "problem resolution"],
    constraints=[
        "Cannot process refunds over $500 without manager approval",
        "Must follow company return policy",
        "Always maintain professional boundaries"
    ],
    response_length="moderate"
)

# Generate system prompt
service_prompt = customer_service.to_system_prompt()
print(service_prompt)

You are ServicePro, a Customer Service Representative.

Your expertise level is intermediate.

You communicate in a conversational style.

Your tone is friendly, empathetic, professional.

Your areas of expertise include: Product information, Order processing, Returns and refunds, Account management.

You prioritize: customer satisfaction, empathy, problem resolution.

You must respect these constraints:
- Cannot process refunds over $500 without manager approval
- Must follow company return policy
- Always maintain professional boundaries

Provide moderate responses.


This persona balances friendliness with professionalism:
- `INTERMEDIATE` expertise handles most customer inquiries
- `CONVERSATIONAL` style creates natural dialogue
- Multiple tone attributes create a warm yet professional interaction
- Domain knowledge covers typical customer service scenarios
- Constraints define clear boundaries and escalation rules
- `simple` language ensures accessibility for all customers

## Dynamic persona selection - Context-based selection
In production systems, we often need to select the appropriate persona based on user context, task type, or user expertise level. Let's build a persona manager that handles dynamic selection.

### Creating a persona manager
First, let's create a simple persona manager class.

In [12]:
class PersonaManager:
    """Manages multiple personas and handles dynamic selection."""
    
    def __init__(self):
        self.personas: Dict[str, PersonaConfig] = {}
        self.logger = logging.getLogger(__name__)
    
    def register_persona(self, persona_id: str, persona: PersonaConfig):
        """Register a persona with a unique identifier."""
        self.personas[persona_id] = persona
        self.logger.info(f"Registered persona: {persona_id} ({persona.name})")
    
    def get_persona(self, persona_id: str) -> Optional[PersonaConfig]:
        """Retrieve a persona by its identifier."""
        return self.personas.get(persona_id)

The `PersonaManager` class provides:
- A dictionary to store personas with unique identifiers
- `register_persona()` to add new personas to the system
- `get_persona()` to retrieve personas by ID
- Logging to track persona registration

This basic structure allows us to manage multiple personas in a single system.

### Adding context-aware selection
Now let's add a method that selects personas based on user context.

In [13]:
class PersonaManager:
    """Manages multiple personas and handles dynamic selection."""
    
    def __init__(self):
        self.personas: Dict[str, PersonaConfig] = {}
        self.logger = logging.getLogger(__name__)
    
    def register_persona(self, persona_id: str, persona: PersonaConfig):
        """Register a persona with a unique identifier."""
        self.personas[persona_id] = persona
        self.logger.info(f"Registered persona: {persona_id} ({persona.name})")
    
    def get_persona(self, persona_id: str) -> Optional[PersonaConfig]:
        """Retrieve a persona by its identifier."""
        return self.personas.get(persona_id)
    
    ##########################  select_persona  ##########################

    def select_persona(self, 
                      user_expertise: str = "intermediate",
                      task_type: str = "general",
                      user_preferences: Optional[Dict] = None) -> PersonaConfig:
        """Select appropriate persona based on context."""
        
        # Default to first available persona
        if not self.personas:
            raise ValueError("No personas registered")
        
        # Simple selection logic based on task type
        if task_type == "education":
            persona_id = "tutor"
        elif task_type == "technical_support":
            persona_id = "tech_support"
        elif task_type == "customer_service":
            persona_id = "customer_service"
        else:
            persona_id = list(self.personas.keys())[0]
        
        # Get the selected persona
        persona = self.get_persona(persona_id)
        
        if persona is None:
            # Fallback to first available persona
            persona = list(self.personas.values())[0]
            self.logger.warning(f"Persona {persona_id} not found, using fallback: {persona.name}")
        else:
            self.logger.info(f"Selected persona: {persona.name} for task: {task_type}")
        
        return persona

The `select_persona()` method:
1. Takes context parameters: user expertise, task type, and preferences
2. Uses simple logic to map task types to persona IDs
3. Retrieves the appropriate persona from the registry
4. Falls back to the first available persona if the requested one doesn't exist
5. Logs selection decisions for monitoring

This provides a foundation for more sophisticated selection logic.

### Implementing persona adaptation
Let's add the ability to adapt personas based on user feedback.

In [14]:
class PersonaManager:
    """Manages multiple personas and handles dynamic selection."""
    
    def __init__(self):
        self.personas: Dict[str, PersonaConfig] = {}
        self.logger = logging.getLogger(__name__)
    
    def register_persona(self, persona_id: str, persona: PersonaConfig):
        """Register a persona with a unique identifier."""
        self.personas[persona_id] = persona
        self.logger.info(f"Registered persona: {persona_id} ({persona.name})")
    
    def get_persona(self, persona_id: str) -> Optional[PersonaConfig]:
        """Retrieve a persona by its identifier."""
        return self.personas.get(persona_id)
    
    def select_persona(self, 
                      user_expertise: str = "intermediate",
                      task_type: str = "general",
                      user_preferences: Optional[Dict] = None) -> PersonaConfig:
        """Select appropriate persona based on context."""
        
        # Default to first available persona
        if not self.personas:
            raise ValueError("No personas registered")
        
        # Simple selection logic based on task type
        if task_type == "education":
            persona_id = "tutor"
        elif task_type == "technical_support":
            persona_id = "tech_support"
        elif task_type == "customer_service":
            persona_id = "customer_service"
        else:
            persona_id = list(self.personas.keys())[0]
        
        # Get the selected persona
        persona = self.get_persona(persona_id)
        
        if persona is None:
            # Fallback to first available persona
            persona = list(self.personas.values())[0]
            self.logger.warning(f"Persona {persona_id} not found, using fallback: {persona.name}")
        else:
            self.logger.info(f"Selected persona: {persona.name} for task: {task_type}")
        
        return persona

    ##########################  adapt_persona  ##########################
    
    def adapt_persona(self, 
                     base_persona: PersonaConfig,
                     user_feedback: str) -> PersonaConfig:
        """Adapt a persona based on user feedback."""
        
        # Create a copy to avoid modifying the original
        adapted_persona = deepcopy(base_persona)
        
        # Adjust based on feedback keywords
        feedback_lower = user_feedback.lower()
        
        if "simpler" in feedback_lower or "easier" in feedback_lower:
            adapted_persona.language_complexity = "simple"
            adapted_persona.response_length = "concise"
            self.logger.info("Adapted persona for simpler language")
        
        elif "more detail" in feedback_lower or "explain more" in feedback_lower:
            adapted_persona.response_length = "detailed"
            self.logger.info("Adapted persona for more detailed responses")
        
        elif "formal" in feedback_lower:
            adapted_persona.communication_style = CommunicationStyle.FORMAL
            self.logger.info("Adapted persona for formal communication")
        
        elif "casual" in feedback_lower or "friendly" in feedback_lower:
            adapted_persona.communication_style = CommunicationStyle.CASUAL
            if ToneAttribute.FRIENDLY not in adapted_persona.tone_attributes:
                adapted_persona.tone_attributes.append(ToneAttribute.FRIENDLY)
            self.logger.info("Adapted persona for casual communication")
        
        return adapted_persona

The `adapt_persona()` method:
1. Creates a deep copy of the base persona to preserve the original
2. Analyzes user feedback for keywords indicating desired changes
3. Adjusts persona attributes based on feedback:
   - "simpler" → reduces language complexity and response length
   - "more detail" → increases response length
   - "formal" → switches to formal communication style
   - "casual" → switches to casual style and adds friendly tone
4. Logs adaptation decisions for monitoring
5. Returns the adapted persona

This allows the system to dynamically adjust to user preferences.

### Creating a complete chat chain with persona
Now let's integrate our persona system with LangChain to create a complete chat chain.

In [15]:
class PersonaManager:
    """Manages multiple personas and handles dynamic selection."""
    
    def __init__(self):
        self.personas: Dict[str, PersonaConfig] = {}
        self.logger = logging.getLogger(__name__)
    
    def register_persona(self, persona_id: str, persona: PersonaConfig):
        """Register a persona with a unique identifier."""
        self.personas[persona_id] = persona
        self.logger.info(f"Registered persona: {persona_id} ({persona.name})")
    
    def get_persona(self, persona_id: str) -> Optional[PersonaConfig]:
        """Retrieve a persona by its identifier."""
        return self.personas.get(persona_id)
    
    def select_persona(self, 
                      user_expertise: str = "intermediate",
                      task_type: str = "general",
                      user_preferences: Optional[Dict] = None) -> PersonaConfig:
        """Select appropriate persona based on context."""
        
        # Default to first available persona
        if not self.personas:
            raise ValueError("No personas registered")
        
        # Simple selection logic based on task type
        if task_type == "education":
            persona_id = "tutor"
        elif task_type == "technical_support":
            persona_id = "tech_support"
        elif task_type == "customer_service":
            persona_id = "customer_service"
        else:
            persona_id = list(self.personas.keys())[0]
        
        # Get the selected persona
        persona = self.get_persona(persona_id)
        
        if persona is None:
            # Fallback to first available persona
            persona = list(self.personas.values())[0]
            self.logger.warning(f"Persona {persona_id} not found, using fallback: {persona.name}")
        else:
            self.logger.info(f"Selected persona: {persona.name} for task: {task_type}")
        
        return persona
    
    def adapt_persona(self, 
                     base_persona: PersonaConfig,
                     user_feedback: str) -> PersonaConfig:
        """Adapt a persona based on user feedback."""
        
        # Create a copy to avoid modifying the original
        adapted_persona = deepcopy(base_persona)
        
        # Adjust based on feedback keywords
        feedback_lower = user_feedback.lower()
        
        if "simpler" in feedback_lower or "easier" in feedback_lower:
            adapted_persona.language_complexity = "simple"
            adapted_persona.response_length = "concise"
            self.logger.info("Adapted persona for simpler language")
        
        elif "more detail" in feedback_lower or "explain more" in feedback_lower:
            adapted_persona.response_length = "detailed"
            self.logger.info("Adapted persona for more detailed responses")
        
        elif "formal" in feedback_lower:
            adapted_persona.communication_style = CommunicationStyle.FORMAL
            self.logger.info("Adapted persona for formal communication")
        
        elif "casual" in feedback_lower or "friendly" in feedback_lower:
            adapted_persona.communication_style = CommunicationStyle.CASUAL
            if ToneAttribute.FRIENDLY not in adapted_persona.tone_attributes:
                adapted_persona.tone_attributes.append(ToneAttribute.FRIENDLY)
            self.logger.info("Adapted persona for casual communication")
        
        return adapted_persona

    ##########################  create_chat_chain  ##########################
    
    def create_chat_chain(self, persona: PersonaConfig, model_name: str = "gpt-4o-mini-2024-07-18"):
        """Create a LangChain chat chain with the specified persona."""
        
        # Initialize the LLM
        llm = ChatOpenAI(model=model_name, temperature=0.7, openai_api_key=openai_api_key)
        
        # Create prompt template with persona system prompt
        prompt = ChatPromptTemplate.from_messages([
            ("system", persona.to_system_prompt()),
            ("human", "{user_input}")
        ])
        
        # Create the chain
        chain = prompt | llm
        
        self.logger.info(f"Created chat chain with persona: {persona.name}")
        
        return chain

The `create_chat_chain()` method:
1. Initializes a ChatOpenAI LLM with specified model
2. Creates a ChatPromptTemplate with:
   - System message containing the persona's system prompt
   - Human message placeholder for user input
3. Chains the prompt and LLM together using the `|` operator
4. Returns the complete chain ready for use

This integrates our structured persona configuration with LangChain's chat functionality.

## Complete working example

### Building a multi-persona system
Let's put everything together in a complete working example.

In [16]:
# Configure logging
logging.basicConfig(level=logging.INFO)

# Create persona manager
manager = PersonaManager()

# Register our three personas
manager.register_persona("tutor", educational_tutor)
manager.register_persona("tech_support", tech_support)
manager.register_persona("customer_service", customer_service)

# Example 1: Select persona for educational task
edu_persona = manager.select_persona(
    user_expertise="beginner",
    task_type="education"
)
print(f"Selected persona: {edu_persona.name}")
print(f"System prompt:\n{edu_persona.to_system_prompt()}")

# Example 2: Adapt persona based on user feedback
user_feedback = "Can you explain this in simpler terms?"
adapted_persona = manager.adapt_persona(edu_persona, user_feedback)
print(f"\nAdapted persona system prompt:\n{adapted_persona.to_system_prompt()}")

# Example 3: Create chat chain with adapted persona
chat_chain = manager.create_chat_chain(adapted_persona)

# Use the chain
response = chat_chain.invoke({"user_input": "What is machine learning?"})
print(f"\nAgent response:\n{response.content}")

INFO:__main__:Registered persona: tutor (Alex the Tutor)
INFO:__main__:Registered persona: tech_support (TechHelper)
INFO:__main__:Registered persona: customer_service (ServicePro)
INFO:__main__:Selected persona: Alex the Tutor for task: education
INFO:__main__:Adapted persona for simpler language


Selected persona: Alex the Tutor
System prompt:
You are Alex the Tutor, a Educational AI Tutor.

Your expertise level is adaptive.

You communicate in a educational style.

Your tone is friendly, patient, enthusiastic.

Your areas of expertise include: Mathematics, Science, Programming, Study Skills.

You prioritize: clarity, encouragement, accuracy, student success.

You must respect these constraints:
- Never provide direct answers to homework problems
- Always encourage critical thinking
- Adapt explanations to student's demonstrated level

Provide detailed responses.

Adapted persona system prompt:
You are Alex the Tutor, a Educational AI Tutor.

Your expertise level is adaptive.

You communicate in a educational style.

Your tone is friendly, patient, enthusiastic.

Your areas of expertise include: Mathematics, Science, Programming, Study Skills.

You prioritize: clarity, encouragement, accuracy, student success.

You must respect these constraints:
- Never provide direct answers 

INFO:__main__:Created chat chain with persona: Alex the Tutor
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Agent response:
Great question! Machine learning is a fascinating area of computer science where computers learn from data and improve their performance on a task over time without being explicitly programmed for that task. 

Here's a simple way to think about it:

1. **Data**: Machine learning relies on large sets of data. This data can come in various forms, such as numbers, text, or images.

2. **Learning**: The computer uses algorithms to find patterns or relationships in the data. This is similar to how we learn from experience.

3. **Prediction**: Once the machine has learned from the data, it can make predictions or decisions based on new, unseen data.

Can you think of any examples where machine learning might be used in everyday life?


This complete example demonstrates:
1. Setting up logging for monitoring
2. Creating a PersonaManager instance
3. Registering multiple personas with unique IDs
4. Selecting a persona based on task type and user expertise
5. Adapting the persona based on user feedback
6. Creating a LangChain chat chain with the adapted persona
7. Using the chain to generate responses

The system maintains consistency while allowing dynamic adaptation.